In [53]:
import pandas as pd
import cvxpy as cp
import numpy as np

In [54]:
df = pd.read_csv("~/Desktop/OSU/OSU Senior/Fall/ISE 3230/poi_and_kiosk_by_ct_data.csv")

In [81]:
cost_map = {
    5 : 1500, # Bronx
    47 : 1000, # Brooklyn
    61 : 1250, # Manhattan
    81 : 1500, # Quenns
    85 : 1100, # Staten Island
}

df['Cost'] = df['COUNTYFP'].map(cost_map)

In [125]:
# Get vectors and change nan pois to 0
geoid = df['GEOID']
kiosks=df['kiosks'].to_numpy()
pop=df['population'].to_numpy()
pois=df['pois'].to_numpy()
cost=df['Cost'].to_numpy()
borough=df['COUNTYFP']

pois = np.nan_to_num(pois, nan=0, posinf=1, neginf=0)

In [126]:
# Constants
n = len(geoid)
min_pois = 15
max_kiosks = 2000
min_pop = 10000
max_ct_kiosks = 25
borough_pop = 5000

In [127]:
X = cp.Variable(n, integer=True)

# Minimize total cost
obj_fun = cp.sum(cp.multiply(cost, X))

    
constraints = []
constraints.append(X>=0)
constraints.append(X<=max_kiosks) # Can have maximum of 2000 kiosks total ($2 million max budget)
constraints.append(X <= pois/min_pois*5) # For every 15 pois can have max 5 kiosks in CT
constraints.append(X <= pop/min_pop*10) # For every 3000 people can have max 7 kiosks CT

for b in [5, 47, 61, 81, 85]: # For Each borough
    lt = len(geoid[borough==b]) # numer of CT in burough
    # average population in the borough divided by 5000 <= average total kiosks
    # On average over each CT in a borough 1 total kiosk for 5000 people
    constraints.append(sum(pop[borough==b]/borough_pop)/lt <= cp.sum(kiosks[borough==b]+X[borough==b])/lt)



problem = cp.Problem(cp.Minimize(obj_fun), constraints)
problem.solve(solver=cp.GUROBI, verbose=True)


print("Solver status:", problem.status)
print("X values:", X.value)
print("Objective Function", obj_fun.value)


                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 25 02:50:14 PM: Your problem has 2327 variables, 9313 constraints, and 0 parameters.
(CVXPY) Nov 25 02:50:14 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 02:50:14 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 02:50:14 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 25 02:50:14 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 02:50:14 PM: Compiling problem (target solver=GUROBI

In [131]:
print("Total Kiosks added: ", sum(X.value))
print("Total cost: ", obj_fun.value)

for b in [5, 47, 61, 81, 85]:  # Adjust number of boroughs as needed
    # Sum kiosks added in this borough
    kiosks_in_borough = np.sum(X.value[borough == b])
    cost_for_b = np.sum(np.multiply(cost[borough==b], X.value[borough==b]))
    print(f"Borough {b} : Kiosks added = {kiosks_in_borough}")
    print(f"Borough {b} : Cost = {cost_for_b}")


Total Kiosks added:  459.0
Total cost:  566900.0
Borough 5 : Kiosks added = 82.0
Borough 5 : Cost = 123000.0
Borough 47 : Kiosks added = 204.0
Borough 47 : Cost = 204000.0
Borough 61 : Kiosks added = 0.0
Borough 61 : Cost = 0.0
Borough 81 : Kiosks added = 124.0
Borough 81 : Cost = 186000.0
Borough 85 : Kiosks added = 49.0
Borough 85 : Cost = 53900.0


In [132]:
df['kiosks_added'] = X.value

In [133]:
df.to_csv("after_optimization_kiosks.csv")